In [1]:
import sys
sys.path.append("/Users/tmorzadec/Missions/MAPIE")
import numpy as np
from mapie.utils import masked_quantile
from mapie.regression import MapieRegressor
from sklearn.datasets import make_regression
import numpy.ma as ma
from pycallgraph2 import PyCallGraph, Config
from pycallgraph2.output import GraphvizOutput
from functools import lru_cache
import callgraph.decorator as callgraph
%load_ext autoreload
%autoreload 2

In [6]:
X = np.random.uniform(low=-100, high=100, size = int(1e8)).reshape(int(1e4), -1)

# indices1 = np.random.choice(X.shape[0], size=1, replace=True)
# indices2 = np.random.choice(X.shape[1], size=1, replace=True)
# indices = zip(indices1, indices2)

# for (x, y) in indices:
#     X[x,y] = np.nan
#                      

In [3]:
q = list(np.linspace(0.1, 0.9, 1000))
print(X.shape)

(1000, 10000)


In [4]:
@callgraph()
@lru_cache()
def mask():
    masked_quantile(ma.masked_invalid(X), q, axis=0, method="higher")

In [ ]:
%timeit -n 1 -r 1 masked_quantile(ma.masked_invalid(X), q, axis=0, method="higher")

In [9]:
%timeit np.nanquantile(X, q, axis=0, interpolation="higher")

7.48 s ± 401 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit np.quantile(X, q, axis=0, interpolation="higher")

7.35 s ± 502 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
config = Config(max_depth=10)
with PyCallGraph(output=GraphvizOutput(), config=config):
    masked_quantile(ma.masked_invalid(X), q, axis=0, method="higher")


In [ ]:
mapie_reg = MapieRegressor(method="minmax", agg_function="mean", cv=-1)
alpha = [0.2, 0.8]
mapie_reg.fit(X, y)
#y_pred_float1, y_pis_float1 = mapie_reg.predict(X, alpha=alpha[0])
#y_pred_float2, y_pis_float2 = mapie_reg.predict(X, alpha=alpha[1])
y_pred_array, y_pis_array = mapie_reg.predict(X, alpha=alpha)
#print(y_pis_float1[0,1,:10])
#print(y_pis_float2[0,1,:10])
print(y_pis_array[0,1,:10])
print(y_pis_array.shape)

2
500
(1, 500)
2
500
(1, 500)
2
500
(1, 500)
2
500
(1, 500)
[-118.90258408  209.02916237  -22.18296168  234.40885886 -159.61745268
 -184.84111207   26.85546511  -19.12965127 -162.97950999  -53.36413493]
(1, 2, 1000)


In [ ]:

X[0,:].flatten()

array([-0.44901419, -0.31701145,  0.03759213, -1.03072401,  0.32107287,
        0.73329445, -0.16373546, -0.58167561,  0.24257418, -0.40065236])